In [3]:
%load_ext autoreload
%autoreload 2
import os, sys
sys.path.insert(0, os.path.abspath(".."))   

import numpy as np
    
from src.coco.gen.dag_confounded import DAGConfounded
from src.coco.gen.dag_gen import _random_nonlinearity
from src.coco.util.mi_sampling import Sampler
from src.coco.coco import CoCo


def _show(res, method, id):
    tp, fp, tn, fn, f1, _, _ = res
    print(f'\t{method}-{id}: \t(f1={np.round(f1, 2)})'
          f'\t(tp={tp}, tn={tn}, fp={fp}, fn={fn})')




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:

# parameters to test
n_nodes, n_confounders, n_contexts, n_samples = (5, 1, 5, 500)
n_shifts_observed, n_shifts_confounders = 1, 1
fun_form = _random_nonlinearity()

# hyperparameters
seed = 42
FCI_INDEP_TEST = 'kci' # 'fisherz'

# Data Generation
sampler = Sampler()
dag = DAGConfounded(seed, n_contexts, n_nodes, n_confounders, n_shifts_observed, n_shifts_confounders,
                    is_bivariate=False)

D, Dobs = dag.gen_data(seed, n_samples, _functional_form=fun_form, oracle_partition=True, noise_iv=False)


# with known number of confounders
coco_oZ = CoCo(D, dag.G.nodes, sampler=sampler, n_components=n_confounders, dag=dag)
_show(coco_oZ.eval_estimated_edges(dag), coco_oZ, 'cfd')

# with unknown number of confounders
coco = CoCo(D, dag.G.nodes, sampler=sampler, n_components=None, dag=dag, verbosity=0)
_show(coco.eval_estimated_edges(dag), coco, 'cfd')

res = coco.eval_causal(dag)
res = res[0], res[1], res[2], res[3], res[4], 0, 0

_show(res, coco, 'caus')


from src.coco.util.co_test_types import CoShiftTestType, CoDAGType

# CoCo with known causal directions
coco_oG = CoCo(D, dag.G.nodes, sampler=sampler, n_components=None,
               dag_discovery=CoDAGType.SKIP, dag=dag)
_show(coco_oG.eval_estimated_edges(dag), coco_oG, 'cfd')

# CoCo with known causal directions as well as partitions for each node
coco_oPi = CoCo(D, dag.G.nodes, sampler=sampler, n_components=None,
                shift_test=CoShiftTestType.SKIP,
                dag_discovery=CoDAGType.SKIP, dag=dag)

_show(coco_oPi.eval_estimated_edges(dag), coco_oPi, 'cfd')

from src.exp.exp_coco.method_types import MethodType
from src.exp.exp_coco import run_fci

fci_jci, D, Dobs, seed = run_fci(dag, D, Dobs, n_samples, fun_form, MethodType.FCI_JCI, FCI_INDEP_TEST, seed)

tp, fp, tn, fn, f1 = fci_jci.eval_confounded(dag, MethodType.FCI_JCI)
print(f'\tJFCI-cfd: \t(f1={np.round(f1, 2)})\t(tp={tp}, tn={tn}, fp={fp}, fn={fn})')

tp, fp, tn, fn, f1 = fci_jci.eval_causal(dag, MethodType.FCI_JCI)
print(f'\tJFCI-caus: \t(f1={np.round(f1, 2)})\t(tp={tp}, tn={tn}, fp={fp}, fn={fn})')



	coco_oZ-cfd: 	(f1=0.29)	(tp=1, tn=4, fp=0, fn=5)
	coco-cfd: 	(f1=0.29)	(tp=1, tn=4, fp=0, fn=5)
	coco-caus: 	(f1=0.6)	(tp=3, tn=13, fp=2, fn=2)
	coco_oG-cfd: 	(f1=0.67)	(tp=3, tn=4, fp=0, fn=3)
	coco_oG_oPi-cfd: 	(f1=0.0)	(tp=0, tn=4, fp=0, fn=6)


ImportError: cannot import name 'run_fci' from 'src.exp.exp_coco' (/workspace/src/exp/exp_coco/__init__.py)